In [1]:
import random

import numpy as np
import tensorflow.compat.v1 as tf
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras import layers, Sequential, Input
from sklearn.model_selection import train_test_split
from generator import Generator
from keras import metrics
import importlib
import inverse
from matplotlib.ticker import PercentFormatter

In [2]:
def custom_loss(y_true, y_pred):
    pass

In [3]:
import tools.reportParser as parser
dataset, paths = parser.parse_all_files('data/reports/')

C:\Users\ivano\PycharmProjects\isotherm\tools\reportParser.py:34: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv(io.StringIO(value), sep="\s+|\t+|\s+\t+|\t+\s+")
C:\Users\ivano\PycharmProjects\isotherm\tools\reportParser.py:34: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv(io.StringIO(value), sep="\s+|\t+|\s+\t+|\t+\s+")
C:\Users\ivano\PycharmProjects\isotherm\tools\reportParser.py:34: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you

In [450]:
model = tf.keras.models.load_model('data/models/silica_CNN3_transfer.keras', custom_objects={'abs': tf.math.abs, 'custom_loss': custom_loss})
#model = tf.keras.models.load_model('data/models/silica_best_tyhanov.keras', custom_objects={'abs': tf.math.abs, 'custom_loss': custom_loss})

In [4]:
cut_dataset = []
pressures = np.load("data/initial kernels/Pressure_Silica.npy")
for i, iso in enumerate(dataset):
    if iso['p_adsorption'][0] > pressures[77] or iso['p_adsorption'][-1] < pressures[-10]:
        continue
    cut_dataset.append(iso)

In [6]:
pressures = np.load("data/initial kernels/Pressure_Silica.npy")[77:-10]
pore_widths = np.load("data/initial kernels/Size_Kernel_Silica_Adsorption.npy")
kernel = np.load("data/initial kernels/Kernel_Silica_Adsorption.npy")

In [7]:
isotherms = []
for iso in cut_dataset:
    isotherms.append(np.interp(pressures, iso['p_adsorption'], iso['adsorption']))

In [48]:
isotherms[0].shape

(371,)

In [9]:
scales=[]
for i in range(len(isotherms)):
    scale = max(isotherms[i])
    isotherms[i] = isotherms[i] / scale
    scales.append(scale)

In [451]:
#prediction = model.predict(np.array(isotherms))
prediction = model.predict(np.array(text_x))

3/3 [==============================] - 0s 10ms/step


In [50]:
prediction = [i[:128] for i in prediction]

In [452]:
import random
i = random.randint(0, len(prediction))
plt.plot(pore_widths, prediction[i], marker=".")
restored_isotherm = np.dot(kernel.T, prediction[i])
plt.show()

plt.plot(pressures, restored_isotherm[77:-10], marker=".")
plt.plot(pressures, text_x[i], marker="*")
plt.show()

In [453]:
def calculate_error(isotherm_real, restored_isotherm):
    defined_error = np.sum(np.abs(restored_isotherm - isotherm_real[77:-10]) / isotherm_real[77:-10]) / len(isotherm_real[77:-10])
    return defined_error


error_lst = []
for i in range(len(prediction)):
    # restored_isotherm = np.dot(kernel.T, prediction[i])
    # error_lst.append(calculate_error(restored_isotherm, isotherms[i])*100)
    restored_isotherm = np.dot(kernel.T, prediction[i])
    error_lst.append(calculate_error(restored_isotherm, np.array(text_x[i]))*100)

In [245]:
plt.hist(error_lst, bins=100)
plt.show()

In [446]:
model_tyhanov = tf.keras.models.load_model('data/models/silica_CNN2_transfer.keras', custom_objects={'abs': tf.math.abs, 'custom_loss': custom_loss})
test = np.load("data/datasets/tyhanov_test.npz")
text_x = test["x_test"]
text_y = test["y_test"]
prediction_tyhanov = model_tyhanov.predict(np.array(text_x))
prediction_tyhanov = [i[:128] for i in prediction_tyhanov]
error_lst_tyhanov = []
for i in range(len(text_x)):
    restored_isotherm = np.dot(kernel.T, prediction_tyhanov[i])
    error_lst_tyhanov.append(calculate_error(restored_isotherm, np.array(text_x[i]))*100)
error_lst_tyhanov = np.sort(error_lst_tyhanov)

3/3 [==============================] - 0s 6ms/step


In [448]:
import random
i = random.randint(0, len(prediction_tyhanov)-1)
plt.plot(pore_widths, prediction_tyhanov[i], marker=".")
restored_isotherm = np.dot(kernel.T, prediction_tyhanov[i])
plt.show()

plt.plot(pressures, restored_isotherm[77:-10], marker=".")
plt.plot(pressures, text_x[i], marker="*")
plt.show()

In [308]:
plt.hist(error_lst_tyhanov, bins=100)
plt.show()

In [19]:
error_lst_math = []
math_kernel = kernel[:, 77:-10]
for i in range(len(prediction)):
    math_PSD = inverse.fit_linear(isotherms[i], math_kernel, 0).x
    restored_isotherm = np.dot(kernel.T, math_PSD)
    error_lst_math.append(calculate_error(restored_isotherm, isotherms[i])*100)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [81]:
plt.hist(error_lst_math, bins=15, alpha=0.5, label='Mathematical solution',
         weights=np.ones(len(error_lst_math)) / len(error_lst_math), range=[0, 40], edgecolor="k")
plt.hist(error_lst, bins=15, alpha=0.5, label='Model gauss',
         weights=np.ones(len(error_lst)) / len(error_lst), range=[0, 40], edgecolor="k")
plt.hist(error_lst_tyhanov, bins=15, alpha=0.5, label='Model real PSD',
         weights=np.ones(len(error_lst_tyhanov)) / len(error_lst_tyhanov), range=[0, 40], edgecolor="k")
plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
plt.legend(loc='upper right')
plt.ylabel("")
plt.xlabel("Fitted error")
plt.show()

In [80]:
plt.hist(error_lst_math, bins=15, alpha=0.5, label='Mathematical solution',
         weights=np.ones(len(error_lst_math)) / len(error_lst_math), range=[0, 40], edgecolor="k")
plt.hist(error_lst, bins=15, alpha=0.5, label='Model gauss',
         weights=np.ones(len(error_lst)) / len(error_lst), range=[0, 40], edgecolor="k")
plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
plt.legend(loc='upper right')
plt.ylabel("")
plt.xlabel("Fitted error")
plt.show()

In [79]:
plt.hist(error_lst_math, bins=15, alpha=0.5, label='Mathematical solution',
         weights=np.ones(len(error_lst_math)) / len(error_lst_math), range=[0, 40], edgecolor="k")
plt.hist(error_lst_tyhanov, bins=15, alpha=0.5, label='Model real PSD',
         weights=np.ones(len(error_lst_tyhanov)) / len(error_lst_tyhanov), range=[0, 40], edgecolor="k")
plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
plt.legend(loc='upper right')
plt.ylabel("")
plt.xlabel("Fitted error")
plt.show()

In [23]:
plt.hist(error_lst, bins=15, alpha=0.5, label='Model gauss',
         weights=np.ones(len(error_lst)) / len(error_lst), range=[0, 40], edgecolor="k")
plt.hist(error_lst_tyhanov, bins=15, alpha=0.5, label='Model real PSD',
         weights=np.ones(len(error_lst_tyhanov)) / len(error_lst_tyhanov), range=[0, 40], edgecolor="k")
plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
plt.legend(loc='upper right')
plt.ylabel("")
plt.xlabel("Fitted error")
plt.show()

In [22]:
len(isotherms)

609

In [114]:
i = 4
restored_isotherm = np.dot(kernel.T, prediction[i])
math_PSD = inverse.fit_linear(isotherms[i], math_kernel, alpha=0.001).x
restored_isotherm_math = np.dot(kernel.T, math_PSD)

plt.plot(pore_widths, prediction[i], marker=".", label="model")
plt.plot(pore_widths, math_PSD, marker=".", label="math")
plt.legend()
plt.show()

plt.plot(pressures, restored_isotherm[77:-10], marker=".", label="model")
plt.plot(pressures, isotherms[i], marker="*", label="real")
plt.legend()
plt.show()

In [24]:
test = np.load("data/datasets/tyhanov_test.npz")
text_x = test["x_test"]
text_y = test["y_test"]
len(text_x)

92

In [143]:
i = 7
prediction_tyh = model_tyhanov.predict(text_x)
restored_isotherm = np.dot(kernel.T, prediction_tyh[i])
math_PSD = inverse.fit_linear(text_x[i], math_kernel, 0.001).x

plt.plot(pore_widths, prediction_tyh[i], marker=".", label="model")
plt.plot(pore_widths, math_PSD, marker=".", label="math")
plt.legend()
plt.show()

plt.plot(pressures, restored_isotherm[77:-10], marker=".", label="model")
plt.plot(pressures, text_x[i], marker="*", label="real")
plt.legend()
plt.show()

3/3 [==============================] - 0s 11ms/step


In [128]:
exp_file_list = ["MCM-41", "SBA-15", "SBA-16", "MIL-101", "MIL-101_2", "DUT-49", "FDM-4", "PCN-333", "PCN-777",
                 "MIL-100"]

p_exp_list = []
n_s_exp_raw_list = []
for exp_file_name in exp_file_list:
    data = pd.read_csv(f"data/real/{exp_file_name}.txt", header=None)
    # p_exp_list.append(data.iloc[:,1].to_numpy())
    # n_s_exp_raw_list.append(data.iloc[:,3].to_numpy())
    p_exp_list.append(data.iloc[:, 1].to_numpy())
    n_s_exp_raw_list.append(data.iloc[:, 3].to_numpy())
# интерполируем экспериментальную изотерму под давления кернала
n_s_exp_list = []
for i in range(len(p_exp_list)):
    n_s_exp_list.append(np.interp(pressures, p_exp_list[i], n_s_exp_raw_list[i]))

In [133]:
for i in range(len(n_s_exp_list)):
    n_s_exp_list[i] = n_s_exp_list[i]/max(n_s_exp_list[i])

In [135]:
structures_prediction =  model_tyhanov.predict(np.array(n_s_exp_list))

1/1 [==============================] - 0s 43ms/step


In [29]:
structures_prediction = [i[:128] for i in structures_prediction]

In [153]:
i = 6
print(exp_file_list[i])
plt.plot(pore_widths[:-30], structures_prediction[i][:-30], marker=".", label="PSD")
plt.ylabel("Объем пор, $см^3$/$нм*г$")
plt.xlabel("Размер пор, $нм$")
plt.legend(loc='upper right')
plt.grid()
plt.show()

restored_isotherm = np.dot(math_kernel.T, structures_prediction[i])
plt.plot(pressures, restored_isotherm, label="Изотерма по полученному моделью распределению")
plt.plot(pressures, n_s_exp_list[i], label="Экспериментальная изотерма")
plt.ylabel("Адсорбция, $см^3$(НТД)/$г$")
plt.xlabel("Давление, $P/P_{0}$")
plt.legend(loc='upper left')
plt.grid()
plt.show()

FDM-4


In [454]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde


x = np.linspace(0, 40, 1000)

kde_math = gaussian_kde(error_lst_math)
kde_model = gaussian_kde(error_lst)
kde_real = gaussian_kde(error_lst_tyhanov)

pdf_math = kde_math(x)
pdf_model = kde_model(x)
pdf_real = kde_real(x)

plt.plot(x, pdf_math, label='Классическое решение', color='blue')
plt.plot(x, pdf_model, label='Модель на основе искусственных распределений', color='orange')
plt.plot(x, pdf_real, label='Модель на основе реальных распределений', color='green')

plt.legend(loc='upper right')
#plt.ylabel(r'Ошибка описания изотермы 1/%')
plt.xlabel("Ошибка описания изотермы %")
plt.title("Распределения ошибок моделей (PDF)")
plt.grid()
plt.show()
